In [1]:
from aicsimageio import AICSImage, imread
from aicsimageio.writers import PngWriter
from pathlib import Path
import napari
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

### CZI file visualisation

In [2]:
%gui qt

In [3]:
ext = 'czi'
# folder = '../../../datasets/microscopy_luke/Movies_SuperRes/Movies_SuperRes/'
# folder = '../../../datasets/microscopy_luke/Movies_SuperRes/Movies_SuperRes/LifeAct Live Imaging/'
folder = '../../../datasets/microscopy_luke/20180412/day5 PKCzT410E+PWPI-mCherry/'

In [4]:
def get_paths(dir_path, ext):
    return [str(path) for path in Path(dir_path).rglob('*.{}'.format(ext))]

In [5]:
files = get_paths(folder, ext)

In [6]:
len(files)

59

In [7]:
# img = AICSImage(files[5])
img = AICSImage('/home/victor/datasets/microscopy_luke/20180620/day5 PKCzT410E+PWPI-mCherry/day5 PKCzT410E+PWPI-mCherry-13.czi')
img = img.get_image_data()
viewer = napari.view_image(img)
img.shape

/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "
/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension V with depth 1, assuming V=0 is the desired value, if not the case specify V=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "


(1, 1, 5, 1, 1024, 1024)

In [29]:
folder = 'samples'
os.makedirs(folder, exist_ok=True)
img = img.squeeze(0).squeeze(0)
for i in range(img.shape[0]):
    im = Image.fromarray(img[i].squeeze(0))
    path = os.path.join(folder, 'channel_{}.png'.format(i))
    im.save(path)

### TIFF file visualisation

In [ ]:
ext_tiff = 'tif'
folder_tiff = '../../datasets/microscopy_luke/20180620/analyze images/control'

In [ ]:
tiff_files = get_paths(folder_tiff, ext_tiff)

In [ ]:
len(tiff_files)

In [ ]:
img = AICSImage(tiff_files[5])

In [ ]:
img.shape

In [ ]:
img = img.get_image_data("CZYX", S=0, T=0)

In [ ]:
viewer = napari.view_image(img)

In [ ]:
img.shape

In [ ]:
img = img.squeeze(1)

In [ ]:
plt.figure()
plt.imshow(img[1], cmap='gray')

In [ ]:
img.shape

In [ ]:
img = np.asarray(img[1])

In [ ]:
img.shape

### Bicubic downsample

In [ ]:
import torch
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image

In [ ]:
trans = transforms.ToTensor()
img_tsr = trans(img)
img_tsr = img_tsr.unsqueeze(0)
img_tsr.shape

In [ ]:
bic_img = F.interpolate(img_tsr, scale_factor=0.125, mode='bicubic')

In [ ]:
bic_img.shape

In [ ]:
np_img = bic_img.squeeze(0).squeeze(0).numpy()

In [ ]:
plt.figure()
plt.imshow(np_img, cmap='gray')

In [ ]:
# build false rgb
img_rgb = np.stack([np_img, np_img, np_img], 2)
img_rgb.shape

In [ ]:
img_rgb = to_pil_image(trans(img_rgb))

In [ ]:
img_rgb

In [ ]:
img_rgb.save('16x_micro.png')